Low amount of data and high number of features, so we need a classification model that has bias/low variance such as Linear SVM.

https://www.kdnuggets.com/2020/05/guide-choose-right-machine-learning-algorithm.html

In [1]:
import os
os.chdir('..')
os.chdir('../resources')
pwd = os.getcwd()

In [2]:
%%time
import pandas as pd
import cv2

features = []
pixels = []
img_names = []
feature_names = ['head', 'spine', 'leg', 'arm', 'hand', 'foot']
for feature in feature_names+ ['none']:
    file_names = os.listdir(pwd+'/features/'+feature)
    img_names = img_names + file_names
    for file in file_names:
        features.append(feature)
        pixels.append(cv2.imread(pwd+'/features/'+feature+'/'+file, 0))

feature_images = pd.DataFrame({'image_name': img_names,'class' : features, 'image_pixels': pixels})

#feature_images = feature_images.groupby('class').head(50)

print('Total features: ', feature_images.shape[0])
print(feature_images.value_counts('class'))
feature_images

Total features:  6731
class
none     4692
head     1366
spine     410
hand      134
arm        81
leg        30
foot       18
dtype: int64
Wall time: 4.07 s


,image_name,class,image_pixels
0,0~19150_2021-01-25_21.00_20523817.png,head,"[[67, 72, 77, 77, 77, 76, 74, 79, 86, 91, 93, ..."
1,1001~20355_2021-07-12_22.38_222558687.png,head,"[[71, 77, 84, 84, 84, 81, 76, 71, 71, 77, 83, ..."
2,1003~20355_2021-07-12_22.38_22314385.png,head,"[[90, 90, 88, 87, 88, 89, 89, 90, 91, 92, 91, ..."
3,1005~20355_2021-07-12_22.54_22427436.png,head,"[[73, 77, 77, 76, 79, 82, 84, 88, 91, 90, 88, ..."
4,1006~20355_2021-07-14_22.51_224458527.png,head,"[[60, 56, 55, 56, 57, 58, 58, 59, 61, 62, 65, ..."
...,...,...,...
6726,6726~20056_2021-02-21_16.51_164223758.png,none,"[[64, 67, 61, 61, 58, 58, 61, 64, 65, 65, 67, ..."
6727,6727~20432_2021-03-16_21.20_211056600.png,none,"[[109, 121, 135, 145, 161, 169, 171, 175, 174,..."
6728,6728~20009_2021-02-07_17.12_171158865.png,none,"[[133, 135, 142, 140, 130, 119, 113, 105, 92, ..."
6729,6729~19575_2021-02-20_10.36_103355493.png,none,"[[76, 77, 76, 76, 72, 73, 77, 81, 85, 87, 89, ..."


In [3]:
def image_size(row):
    row['width'] = row['image_pixels'].shape[0]
    row['height'] = row['image_pixels'].shape[1]
    return row

f = feature_images.apply(image_size, axis=1)
feature_images = f[f['width']>10]
feature_images

,image_name,class,image_pixels,width,height
0,0~19150_2021-01-25_21.00_20523817.png,head,"[[67, 72, 77, 77, 77, 76, 74, 79, 86, 91, 93, ...",124,124
1,1001~20355_2021-07-12_22.38_222558687.png,head,"[[71, 77, 84, 84, 84, 81, 76, 71, 71, 77, 83, ...",190,190
2,1003~20355_2021-07-12_22.38_22314385.png,head,"[[90, 90, 88, 87, 88, 89, 89, 90, 91, 92, 91, ...",156,156
3,1005~20355_2021-07-12_22.54_22427436.png,head,"[[73, 77, 77, 76, 79, 82, 84, 88, 91, 90, 88, ...",184,184
4,1006~20355_2021-07-14_22.51_224458527.png,head,"[[60, 56, 55, 56, 57, 58, 58, 59, 61, 62, 65, ...",163,163
...,...,...,...,...,...
6726,6726~20056_2021-02-21_16.51_164223758.png,none,"[[64, 67, 61, 61, 58, 58, 61, 64, 65, 65, 67, ...",185,185
6727,6727~20432_2021-03-16_21.20_211056600.png,none,"[[109, 121, 135, 145, 161, 169, 171, 175, 174,...",157,157
6728,6728~20009_2021-02-07_17.12_171158865.png,none,"[[133, 135, 142, 140, 130, 119, 113, 105, 92, ...",199,199
6729,6729~19575_2021-02-20_10.36_103355493.png,none,"[[76, 77, 76, 76, 72, 73, 77, 81, 85, 87, 89, ...",53,53


Get images ready for HOG:
1. Filter
2. Resize to (128, 128)

In [4]:
%%time
from Process import process
def resize(x):
    return cv2.resize(x,(128, 128))

feature_images['filtered_pixels'] = feature_images['image_pixels'].apply(process, crop_thresh=0)
feature_images['resized_pixels'] = feature_images['filtered_pixels'].apply(resize)
feature_images

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 1min 40s


,image_name,class,image_pixels,width,height,filtered_pixels,resized_pixels
0,0~19150_2021-01-25_21.00_20523817.png,head,"[[67, 72, 77, 77, 77, 76, 74, 79, 86, 91, 93, ...",124,124,"[[0, 0, 18, 20, 27, 28, 28, 29, 31, 32, 32, 33...","[[0, 0, 17, 20, 26, 28, 28, 29, 30, 31, 32, 33..."
1,1001~20355_2021-07-12_22.38_222558687.png,head,"[[71, 77, 84, 84, 84, 81, 76, 71, 71, 77, 83, ...",190,190,"[[0, 0, 29, 29, 31, 32, 32, 32, 34, 34, 35, 34...","[[7, 23, 29, 31, 32, 33, 34, 34, 34, 34, 34, 3..."
2,1003~20355_2021-07-12_22.38_22314385.png,head,"[[90, 90, 88, 87, 88, 89, 89, 90, 91, 92, 91, ...",156,156,"[[0, 0, 49, 46, 43, 43, 43, 41, 40, 39, 39, 37...","[[6, 20, 47, 43, 43, 42, 40, 39, 39, 37, 34, 3..."
3,1005~20355_2021-07-12_22.54_22427436.png,head,"[[73, 77, 77, 76, 79, 82, 84, 88, 91, 90, 88, ...",184,184,"[[0, 0, 28, 28, 30, 31, 35, 35, 36, 36, 36, 36...","[[6, 20, 28, 30, 34, 35, 36, 36, 36, 36, 37, 3..."
4,1006~20355_2021-07-14_22.51_224458527.png,head,"[[60, 56, 55, 56, 57, 58, 58, 59, 61, 62, 65, ...",163,163,"[[0, 0, 16, 16, 16, 16, 15, 15, 16, 16, 16, 16...","[[2, 8, 16, 16, 16, 15, 16, 16, 16, 18, 21, 26..."
...,...,...,...,...,...,...,...
6726,6726~20056_2021-02-21_16.51_164223758.png,none,"[[64, 67, 61, 61, 58, 58, 61, 64, 65, 65, 67, ...",185,185,"[[0, 0, 143, 143, 143, 143, 143, 136, 136, 124...","[[32, 106, 143, 143, 143, 136, 128, 136, 126, ..."
6727,6727~20432_2021-03-16_21.20_211056600.png,none,"[[109, 121, 135, 145, 161, 169, 171, 175, 174,...",157,157,"[[0, 0, 104, 114, 127, 152, 190, 211, 211, 217...","[[11, 42, 109, 124, 152, 194, 211, 215, 224, 2..."
6728,6728~20009_2021-02-07_17.12_171158865.png,none,"[[133, 135, 142, 140, 130, 119, 113, 105, 92, ...",199,199,"[[0, 0, 96, 96, 96, 96, 98, 98, 98, 96, 90, 86...","[[27, 84, 96, 96, 98, 98, 92, 85, 75, 69, 68, ..."
6729,6729~19575_2021-02-20_10.36_103355493.png,none,"[[76, 77, 76, 76, 72, 73, 77, 81, 85, 87, 89, ...",53,53,"[[0, 0, 26, 26, 26, 26, 26, 26, 26, 32, 41, 43...","[[0, 0, 0, 0, 9, 20, 26, 26, 26, 26, 26, 26, 2..."


Compute HOG features

In [5]:
%%time
from Hog import compute_hog
feature_images.loc[:,'hog'] = feature_images['resized_pixels'].apply(compute_hog)
feature_images

C:\Users\emery\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\emery\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Wall time: 19 s


,image_name,class,image_pixels,width,height,filtered_pixels,resized_pixels,hog
0,0~19150_2021-01-25_21.00_20523817.png,head,"[[67, 72, 77, 77, 77, 76, 74, 79, 86, 91, 93, ...",124,124,"[[0, 0, 18, 20, 27, 28, 28, 29, 31, 32, 32, 33...","[[0, 0, 17, 20, 26, 28, 28, 29, 30, 31, 32, 33...","[[0.04165095580488591, 0.015814938949132096, 0..."
1,1001~20355_2021-07-12_22.38_222558687.png,head,"[[71, 77, 84, 84, 84, 81, 76, 71, 71, 77, 83, ...",190,190,"[[0, 0, 29, 29, 31, 32, 32, 32, 34, 34, 35, 34...","[[7, 23, 29, 31, 32, 33, 34, 34, 34, 34, 34, 3...","[[0.02040571611958285, 0.01862244122099439, 0...."
2,1003~20355_2021-07-12_22.38_22314385.png,head,"[[90, 90, 88, 87, 88, 89, 89, 90, 91, 92, 91, ...",156,156,"[[0, 0, 49, 46, 43, 43, 43, 41, 40, 39, 39, 37...","[[6, 20, 47, 43, 43, 42, 40, 39, 39, 37, 34, 3...","[[0.033062974249889604, 0.027687685818153115, ..."
3,1005~20355_2021-07-12_22.54_22427436.png,head,"[[73, 77, 77, 76, 79, 82, 84, 88, 91, 90, 88, ...",184,184,"[[0, 0, 28, 28, 30, 31, 35, 35, 36, 36, 36, 36...","[[6, 20, 28, 30, 34, 35, 36, 36, 36, 36, 37, 3...","[[0.032707920596178816, 0.011992689289396037, ..."
4,1006~20355_2021-07-14_22.51_224458527.png,head,"[[60, 56, 55, 56, 57, 58, 58, 59, 61, 62, 65, ...",163,163,"[[0, 0, 16, 16, 16, 16, 15, 15, 16, 16, 16, 16...","[[2, 8, 16, 16, 16, 15, 16, 16, 16, 18, 21, 26...","[[0.06231765893353702, 0.011615929122069851, 0..."
...,...,...,...,...,...,...,...,...
6726,6726~20056_2021-02-21_16.51_164223758.png,none,"[[64, 67, 61, 61, 58, 58, 61, 64, 65, 65, 67, ...",185,185,"[[0, 0, 143, 143, 143, 143, 143, 136, 136, 124...","[[32, 106, 143, 143, 143, 136, 128, 136, 126, ...","[[0.12505852356474076, 0.15459917003369625, 0...."
6727,6727~20432_2021-03-16_21.20_211056600.png,none,"[[109, 121, 135, 145, 161, 169, 171, 175, 174,...",157,157,"[[0, 0, 104, 114, 127, 152, 190, 211, 211, 217...","[[11, 42, 109, 124, 152, 194, 211, 215, 224, 2...","[[0.18619225965468383, 0.003477184368232724, 0..."
6728,6728~20009_2021-02-07_17.12_171158865.png,none,"[[133, 135, 142, 140, 130, 119, 113, 105, 92, ...",199,199,"[[0, 0, 96, 96, 96, 96, 98, 98, 98, 96, 90, 86...","[[27, 84, 96, 96, 98, 98, 92, 85, 75, 69, 68, ...","[[0.11487861108932775, 0.05788737573414593, 0...."
6729,6729~19575_2021-02-20_10.36_103355493.png,none,"[[76, 77, 76, 76, 72, 73, 77, 81, 85, 87, 89, ...",53,53,"[[0, 0, 26, 26, 26, 26, 26, 26, 26, 32, 41, 43...","[[0, 0, 0, 0, 9, 20, 26, 26, 26, 26, 26, 26, 2...","[[0.0821274300579101, 0.0334630722485934, 0.02..."


Separate into test and train data

Train Linear SVMs for each feature

In [8]:
%%time
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import cross_val_score

feature_svm = []
fully_trained = []
accuracy = []
classification_rep = []
for feature in feature_names:
    print(feature, end=" ")
    # Have each SVM think that other features are actually 'none'
    df = feature_images.copy(deep=True)
    not_cur = feature_names[:]
    not_cur.remove(feature)
    df['class'] = df['class'].replace(not_cur,'none')
    
    df = df[df['class'].isin([feature, 'none'])]
    
    labels =  np.array(df['class']).reshape(len(df['class']),1)
    hog_features = np.stack(df['hog'], axis=1)[0]
    data = np.hstack((hog_features,labels))

    clf = svm.SVC(probability=True)
    scores = cross_val_score(clf, hog_features, labels.ravel(), cv=2)
    
    print(scores)
    
    clf2 = svm.SVC(probability=True)
    clf2.fit(data[:len(data),:-1],data[:len(data),-1:].ravel())
    
    fully_trained.append(clf2)
    accuracy.append(scores)

results = pd.DataFrame({'feature': feature_names, 'svm': feature_svm, 'accuracy': accuracy})

head [0.95453195 0.97503715]
spine [0.95928678 0.95423477]
leg [0.99554235 0.99554235]
arm [0.98811293 0.98781575]
hand [0.97682021 0.98008915]
foot [0.99732541 0.99732541]


ValueError: arrays must all be same length

Print results

In [9]:
a = pd.DataFrame(accuracy, index=feature_names)
a['mean'] = a.mean(axis=1)
a

,0,1,mean
head,0.954532,0.975037,0.964785
spine,0.959287,0.954235,0.956761
leg,0.995542,0.995542,0.995542
arm,0.988113,0.987816,0.987964
hand,0.976820,0.980089,0.978455
foot,0.997325,0.997325,0.997325


In [11]:
import pickle
from pathlib import Path
svm_folder =  pwd+"/SVM/"
for clf in fully_trained:
    class_ = clf.classes_
    class_ = np.copy(class_).tolist()
    class_.remove('none')
    pkl_filename = class_[0]+'.pkl'
    with open(svm_folder+pkl_filename, 'wb+') as file:
        pickle.dump(clf, file)

[SVC(probability=True), SVC(probability=True), SVC(probability=True), SVC(probability=True), SVC(probability=True), SVC(probability=True)]
